In [1]:
import numpy as np
import random
import copy
import math

# Define constants, variables, and operators
constant_range = np.linspace(0.5, 10, num=25)
CONSTANTS = list(constant_range)

def safe_divide(a, b):
    """
    Safe division that returns 10^6 if the denominator is 0, else normal division.
    """
    if b == 0:
        return float(10**6)  # Return 10^6 (large penalty) when dividing by zero
    result = a / b
    return np.clip(result, -1000, 1000)

def safe_sqrt(a):
    if a < 0:
        return float(10**6)
    result = np.sqrt(a)
    return np.clip(result, -1000, 1000)

def safe_log(a):
    if a <= 0:
        return float(10**6)
    result = np.log(a)
    return np.clip(result, -1000, 1000)

def factorial(a):
    if a < 0 or not a.is_integer():
        return float(10**6)  # Return a large penalty if input is negative or not an integer
    result = math.factorial(int(a))
    return np.clip(result, -1000, 1000)

def safe_exp(x):
    x = np.clip(x, -700, 700)  # Limiting input to prevent overflow
    result = np.exp(x)
    return np.clip(result, -1000, 1000)

def safe_sin(x):
    x = np.clip(x, -1000, 1000)  # Clip the input to avoid excessive values
    result = np.sin(x)
    return np.clip(result, -1000, 1000)

def safe_cos(x):
    x = np.clip(x, -1000, 1000)  # Clip the input to avoid excessive values
    result = np.cos(x)
    return np.clip(result, -1000, 1000)

def safe_sinh(x):
    x = np.clip(x, -100, 100)  # Limiting input to prevent overflow
    result = np.sinh(x)
    return np.clip(result, -1000, 1000)

def safe_cosh(x):
    x = np.clip(x, -100, 100)  # Limiting input to prevent overflow
    result = np.cosh(x)
    return np.clip(result, -1000, 1000)

def safe_tan(x):
    if np.isclose(np.mod(x, np.pi), np.pi / 2):  # tan is undefined at odd multiples of pi/2
        return float(10**6)
    x = np.clip(x, -1000, 1000)  # Clip the input to a manageable range
    result = np.tan(x)
    return np.clip(result, -1000, 1000)

def safe_log10(x):
    if x <= 0:
        return float(10**6)
    result = np.log10(x)
    return np.clip(result, -1000, 1000)

def safe_pow(base, exp):
    base = np.clip(base, -1000, 1000)  # Limiting base
    exp = np.clip(exp, -1000, 1000)    # Limiting exponent
    if base == 0 and exp < 0:
        return float(10**6)  # Return large penalty for invalid operation
    try:
        result = np.power(base, exp)
        return np.clip(result, -1000, 1000)
    except ValueError:
        return float(10**6)

def safe_log2(x):
    if x <= 0:
        return float(10**6)
    result = np.log2(x)
    return np.clip(result, -1000, 1000)

def safe_mod(x, y):
    if y == 0:
        return float(10**6)
    x = np.clip(x, -1000, 1000)
    y = np.clip(y, -1000, 1000)
    result = np.mod(x, y)
    return np.clip(result, -1000, 1000)

def safe_tanh(x):
    x = np.clip(x, -1000, 1000)  # Limiting large inputs to avoid overflow
    result = np.tanh(x)
    return np.clip(result, -1000, 1000)


# The OPERATORS dictionary, including tan
OPERATORS = {
    'add': np.add,
    'sub': np.subtract,
    'mul': np.multiply,
    'div': safe_divide,
    'sin': safe_sin,
    'cos': safe_cos,
    'sqrt': safe_sqrt,
    'log': safe_log,
    'pow': safe_pow,  # Use safe version of power. example: 2^3 = 8
    'exp': safe_exp,
    'tan': safe_tan,  # Ensure 'tan' is added to the dictionary.
    'sinh': safe_sinh,  # Use safe version of sinh. example: sinh(710) = 1.0e308. 
    'cosh': safe_cosh,
    'tanh': safe_tanh,
    'abs': np.abs,
    'log2': safe_log2,
    'log10': safe_log10,  # Use safe version of log10
    'mod': safe_mod # binary operator
    #'fact': factorial,  # Factorial should be handled with care (NOT applicable for non-integer values)
}

class Node:
    def __init__(self, value, is_operator=False):
        """
        Initialize a Node.
        
        :param value: The value of the node (operator, constant, or variable name).
        :param is_operator: True if the node represents an operator, otherwise False.
        """
        self.value = value
        self.is_operator = is_operator
        self.left = None
        self.right = None

    def evaluate(self, variables, depth=0):
        """
        Evaluate the subtree rooted at this node.
        
        :param variables: A dictionary mapping variable names to their values.
        :param depth: The current recursion depth (for debugging).
        :return: The result of evaluating the subtree.
        """
        if depth > 1000:  # Prevent infinite recursion
            return float(100000)  # Return a large penalty value for exceeding the maximum recursion depth
        
        # Check for None values to avoid errors
        if self is None:
            return 0

        if self.is_operator:
            # Unary operator (e.g., sin, cos, tan, sqrt, log, exp)
            if self.value in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10']:
                if self.left is None:
                    raise ValueError(f"Missing left child for operator {self.value}")
                left_value = self.left.evaluate(variables, depth + 1)
                return OPERATORS[self.value](left_value)
            
            # Binary operator (e.g., add, sub, mul, div, pow)
            elif self.value in ['add', 'sub', 'mul', 'div', 'pow', 'mod']:
                if self.left is None or self.right is None:
                    raise ValueError(f"Missing children for operator {self.value}")
                left_value = self.left.evaluate(variables, depth + 1)
                right_value = self.right.evaluate(variables, depth + 1)
                return OPERATORS[self.value](left_value, right_value)
        
        elif self.value in variables:
            # Variable node (e.g., x0, x1, etc.)
            return variables[self.value]
        
        # Constant node (just return the constant value)
        return self.value
    
class Individual:
    def __init__(self, root):
        """
        Initialize an Individual (a tree).
        
        :param root: The root node of the tree.
        """
        self.root = root
        self.fitness_value = None
        self.mse_percentage = None

    def evaluate(self, variables):
        """
        Evaluate the tree.
        
        :param variables: A dictionary mapping variable names to their values.
        :return: The result of evaluating the tree.
        """
        return self.root.evaluate(variables)

    def fitness(self, file_path):
        # Load the data
        data = np.load(file_path)
        x = data['x']
        y = data['y']

        # Initialize variables for prediction
        num_features = x.shape[0]
        variables = {f'x{i}': None for i in range(num_features)}

        # Compute predictions
        y_pred = []
        for i in range(x.shape[1]):  # Iterate over each column
            for j in range(num_features):  # Set variable values for this row
                variables[f'x{j}'] = x[j, i]
            y_pred.append(self.evaluate(variables))
        y_pred = np.array(y_pred)


        # Calculate MSE
        mse = np.mean((y - y_pred) ** 2)
        self.fitness_value = mse
        #if mse is NaN (due to overflow), set it to 10^6 for not consider the tree
        if np.isnan(mse):
            self.fitness_value = float(10**6)

        # Get the range of y values to determine the scale
        y_min = np.min(y)
        y_max = np.max(y)

        # Calculate the MSE percentage relative to the range of y values
        mse_percentage = (mse / (y_max - y_min)) * 100
        self.mse_percentage = mse_percentage


    def __str__(self):
        """
        Return a string representation of the tree.
        """
        return self._str_helper(self.root)

    def _str_helper(self, node, visited=None):
        """Helper function for string representation. Recursively traverse the tree."""
        if visited is None:
            visited = set()  # Initialize visited set for the first call

        # If node is None, return an empty string
        if node is None:
            return ""

        # Check if we've already visited this node to detect cycles. It should not happen in general.
        if id(node) in visited:
            return "(...)  # Cycle detected"
        
        # Mark the current node as visited
        visited.add(id(node))

        if node.is_operator:
            # Handle binary operators (e.g., add, sub, mul, div)
            if node.value in ['add', 'sub', 'mul', 'div', 'pow', 'mod']:
                left_str = self._str_helper(node.left, visited)
                right_str = self._str_helper(node.right, visited)
                return f"({left_str} {node.value} {right_str})"
            
            # Handle unary operators (e.g., sin, cos, tan, sqrt, log, exp)
            elif node.value in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10']:
                left_str = self._str_helper(node.left, visited)
                return f"({node.value} {left_str})"
        
        # For variable and constant nodes, just return their value as a string
        return str(node.value)
    
    def count_nodes(self, node, depth=0, max_depth=1000):
        """Count the number of nodes in the tree while limiting recursion depth."""
        if node is None or depth > max_depth:
            return 0
        if node.is_operator:
            return 1 + self.count_nodes(node.left, depth + 1, max_depth) + self.count_nodes(node.right, depth + 1, max_depth)
        return 1  # It's a leaf node (constant or variable)





In [2]:
# steps:
# Initialize Population: Generate an initial population of random expression trees (individuals).
# Evaluate Fitness: Assess the fitness of each individual by calculating its Mean Squared Error (MSE) against the target dataset.
# Selection: Apply tournament selection to choose parent individuals based on their fitness.
# Recombination (Crossover): Combine pairs of parent individuals to produce offspring through crossover.
# Mutation: Apply mutation to offspring to introduce genetic diversity.
# Replacement: Replace the old population with the new generation of individuals.
# Termination: Repeat the process for a predefined number of generations or until a satisfactory solution is found.

In [3]:
class SymbolicRegression:
    def __init__(self, population_size, num_generations, mutation_rate, crossover_rate, file_path):
        """
        Initialize the Symbolic Regression algorithm.
        
        :param population_size: Number of individuals in the population.
        :param num_generations: Number of generations to evolve.
        :param mutation_rate: Probability of mutation.
        :param crossover_rate: Probability of crossover.
        :param file_path: Path to the .npz file containing 'x' and 'y'.
        """
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.file_path = file_path
        self.population = []
        
        # Load data
        data = np.load(self.file_path)
        self.x_data = data['x']
        self.y_data = data['y']
        self.num_vars = self.x_data.shape[0]  # Number of variables (rows in x)
        self.VARIABLES = [f'x{i}' for i in range(self.num_vars)] # Generate the variable names dynamically

    def create_tree(self, num_nodes, x_data):
        """
        Create a random tree with the specified number of nodes.
        
        :param num_nodes: The number of nodes in the tree.
        :param x_data: The x data from the .npz file to determine the number of variables.
        :return: A random tree as an Individual object.
        """
        # num_vars = x_data.shape[0]  # Number of variables (rows in x)
        # VARIABLES = [f'x{i}' for i in range(num_vars)]  # Generate the variable names dynamically

        def build_tree(nodes_left, is_root=True):
            """Recursively build a tree."""
            if nodes_left == 1:
                if random.random() < 0.5:
                    return Node(random.choice(self.VARIABLES)), 1
                else:
                    return Node(random.choice(CONSTANTS)), 1
            else:
                operator = random.choice(list(OPERATORS.keys()))

                if operator in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10']:
                    right_subtree_size = nodes_left - 1
                    node = Node(operator, is_operator=True)
                    right_child, _ = build_tree(right_subtree_size, is_root=False)
                    node.left = right_child
                    return node, 1 + right_subtree_size
                elif operator in ['add', 'sub', 'mul', 'div','pow', 'mod']:
                    if nodes_left <= 2:
                        left_subtree_size = 1
                        right_subtree_size = 1
                    else:
                        right_subtree_size = random.randint(1, nodes_left - 2)
                        left_subtree_size = nodes_left - 1 - right_subtree_size

                    node = Node(operator, is_operator=True)

                    left_child, left_size = build_tree(left_subtree_size, is_root=False)
                    right_child, right_size = build_tree(right_subtree_size, is_root=False)

                    node.left = left_child
                    node.right = right_child

                    return node, 1 + left_size + right_size

        root_node, _ = build_tree(num_nodes)
        if root_node is None:
            print(f"Warning: Tree creation failed with num_nodes={num_nodes}. Returning None.")
        return Individual(root_node)

    def create_initial_population(self):
        """Create the initial population of individuals."""
        for _ in range(self.population_size):
            num_nodes = random.randint(4, 10)
            individual = self.create_tree(num_nodes=num_nodes, x_data=self.x_data)
            self.population.append(individual)


    def tournament_selection(self, tournament_size):
        """Select parents using tournament selection.""" #Avoid Fitness Recalculation.
        #print("tournament_size: ", tournament_size)
        selected_parents = []
        for _ in range(self.population_size): 
            tournament = random.sample(self.population, tournament_size)  # Randomly select individuals for the tournament
            
            #print("tournament + fitness: ")
            # for ind in tournament:
            #     print(ind.__str__(), ind.fitness_value)

            best_individual = min(tournament, key=lambda ind: ind.fitness_value)  # Select the best (minimum fitness)
            selected_parents.append(best_individual)
            
            # print("selected parents until now: ")
            # for ind in selected_parents:
            #     print(ind.__str__())
        #print(len(selected_parents))
        return selected_parents


    def swap_subtrees(self, node1, node2):
        """
        Swap the values and child nodes of two subtrees rooted at node1 and node2.
        
        :param node1: First node (subtree root).
        :param node2: Second node (subtree root).
        """
        # # Swap the node values (operation or constant value)
        # node1.value, node2.value = node2.value, node1.value

        # # Swap the left and right children of the nodes
        # node1.left, node2.left = node2.left, node1.left
        # node1.right, node2.right = node2.right, node1.right
        # If both nodes are leaf nodes (variables or constants), we can swap them
        if not node1.is_operator and not node2.is_operator:
            node1.value, node2.value = node2.value, node1.value
            return
        # If one node is a leaf and the other is an operator, we should not swap them
        if (not node1.is_operator and node2.is_operator) or (node1.is_operator and not node2.is_operator):
            return
        # If both nodes are operators, check if they are compatible
        if node1.is_operator and node2.is_operator:
            # Binary operator: requires both left and right children
            if node1.value in ['add', 'sub', 'mul', 'div','pow', 'mod'] and node2.value in ['add', 'sub', 'mul', 'div','pow', 'mod']:
                node1.value, node2.value = node2.value, node1.value
                node1.left, node2.left = node2.left, node1.left
                node1.right, node2.right = node2.right, node1.right
            # Unary operator: only requires a left child (operand)
            elif node1.value in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10'] and node2.value in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10']:
                node1.value, node2.value = node2.value, node1.value
                node1.left, node2.left = node2.left, node1.left  # Unary functions only have a left child
            else:
                # If one is a unary operator and the other is a binary operator, do not swap them
                return
            
    def get_random_node(self, node):
        """
        Recursively select a random node in the tree.

        :param node: The current node in the tree.
        :return: A randomly selected node in the subtree.
        """
        if node is None:
            return None

        candidates = [node]
        if node.is_operator:
            if node.left:
                candidates.append(node.left)
            if node.right:
                candidates.append(node.right)

        return random.choice(candidates)

    def get_tree_depth(self,node):
        """
        Get the depth of the tree (maximum level of nested nodes).
        
        :param node: The root node of the tree.
        :return: Depth of the tree.
        """
        if node is None:
            return 0
        return 1 + max(self.get_tree_depth(node.left), self.get_tree_depth(node.right))

    def crossover(self, parent1, parent2, min_depth=3, max_depth=9):
        """
        Perform recombination (crossover) between two parents to create offspring.

        :param parent1: The first parent Individual.
        :param parent2: The second parent Individual.
        :param min_depth: Mxinimum depth allowed for the resulting tree.
        :param max_depth: Maximum depth allowed for the resulting tree.
        :return: A new individual (child) created from crossover.
        """
        # Deep copy the parents to avoid modifying the originals
        child1 = copy.deepcopy(parent1)
        child2 = copy.deepcopy(parent2)

        # Select random crossover points in both trees
        crossover_point1 = self.get_random_node(child1.root)  # Updated to use self
        crossover_point2 = self.get_random_node(child2.root)  # Updated to use self

        # Swap the subtrees at the crossover points
        self.swap_subtrees(crossover_point1, crossover_point2)

        # Check if the resulting tree is within the allowed depth range
        if self.get_tree_depth(child1.root) < min_depth or self.get_tree_depth(child1.root) > max_depth:
            child1 = copy.deepcopy(parent1)  # Revert to parent1 if depth constraint is violated
        if self.get_tree_depth(child2.root) < min_depth or self.get_tree_depth(child2.root) > max_depth:
            child2 = copy.deepcopy(parent2)  # Revert to parent2 if depth constraint is violated

        # we return the two children which are the parents after the exchange of subtrees
        return child1, child2 


    def mutate(self, individual):
        """
        Perform a point mutation on a random node of the individual.
        This mutation changes a random operator or leaf node with another valid operator or leaf.
        The type of the operator (unary or binary) remains the same.

        :param individual: The individual (symbolic expression) to mutate.
        :return: Mutated individual.
        """

        # Select a random node to mutate
        node_to_mutate = self.select_random_node(individual.root)
        
        if node_to_mutate is None:
            return individual  # If no node is selected, return the individual as is

        # If the node to mutate is a leaf (constant or variable), change it to another leaf
        if not node_to_mutate.is_operator:
            # Randomly select a new constant or variable
            if random.random() < 0.5:  # Mutate to a constant
                node_to_mutate.value = random.choice(CONSTANTS)
            else:  # Mutate to a variable
                node_to_mutate.value = random.choice(self.VARIABLES)
        
        # If the node to mutate is an operator (internal node), change it to a valid operator of the same type
        else:
            if node_to_mutate.is_operator:
                # Binary operators: 'add', 'sub', 'mul', 'div'
                if node_to_mutate.value in ['add', 'sub', 'mul', 'div','pow', 'mod']:
                    # Select a random binary operator but not the same one
                    possible_operators = ['add', 'sub', 'mul', 'div','pow', 'mod']
                    possible_operators.remove(node_to_mutate.value)  # Remove the current operator
                    node_to_mutate.value = random.choice(possible_operators)
                
                # Unary operators: 'sin', 'cos', 'sqrt', 'log'
                elif node_to_mutate.value in ['sin', 'cos', 'sqrt', 'log', 'exp','tan', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10']:
                    # Select a random unary operator but not the same one
                    possible_operators = ['sin', 'cos', 'sqrt', 'log','exp','tan', 'sinh', 'cosh', 'tanh', 'abs', 'log2', 'log10']
                    possible_operators.remove(node_to_mutate.value)  # Remove the current operator
                    node_to_mutate.value = random.choice(possible_operators)
        
        return individual  # Return the mutated individual

        


    def select_random_node(self, node):
        if random.random() < 0.5 and node.is_operator:
            return random.choice([node, node.left, node.right])
        return node
    
    

    
    def evolve(self):
        """Evolve the population over multiple generations."""
        # Step 1: Create the initial population
        self.create_initial_population()

        current_generation = 0
        total_generations = self.num_generations  # Initialize the total number of generations

        # #print initial population and the length of population
        # for ind in self.population:
        #     if ind is None:
        #         print("Found None individual")      
        #     #print(ind.__str__())
        # print("length of initial population of generation: ", len(self.population))

        while current_generation < total_generations:
            if current_generation==0:
                tournament_size = 3

            print(f"Starting Generation {current_generation + 1}")

            # Step 2: Evaluate fitness of each tree in the population
            print(f"Length of population of generation {current_generation + 1}: {len(self.population)}")
            for individual in self.population:
                try:
                    individual.fitness(file_path=self.file_path)
                except Exception as e:  # It should not happen, just in case.
                    print(f"Error evaluating fitness for individual: {e}")
                    individual.fitness_value = float(1000000)  # Assign high fitness value for invalid individuals

            # Step 3: Ensure that the population has valid individuals (with more than one node). It should not happen, just in case.
            valid_population = [ind for ind in self.population if ind.count_nodes(ind.root) > 1]
            while len(valid_population) < self.population_size:  # If the population is empty, create new individuals
                print("Creating new individual")
                new_individual = self.create_tree(num_nodes=random.randint(4, 10), x_data=self.x_data)
                if new_individual.count_nodes(new_individual.root) > 1:  # Check if the new individual has more than one node
                    valid_population.append(new_individual)
            self.population = valid_population

            # Step 4: Elitism: Carry the best `elitism_size` individuals to the next generation
            elitism_size = max(1, self.population_size // 10)  # e.g., top 10% of individuals
            best_individuals = sorted(self.population, key=lambda ind: ind.fitness_value)[:elitism_size]
            next_generation = best_individuals[:]

            # Step 5: Create the next generation using tournament selection and crossover
            unique_individuals = set()  # To store unique identifiers for individuals
            for ind in next_generation:  # Add elitism individuals to the unique set
                unique_individuals.add(hash(str(ind)))

            while len(next_generation) < self.population_size:
                parents = self.tournament_selection(tournament_size)  # Same size of the population
                parent1, parent2 = random.sample(parents, 2)  # Use two parents among the parents selected with tournament selection

                # Apply crossover
                if random.random() < self.crossover_rate:
                    offspring1, offspring2 = self.crossover(parent1, parent2)
                    for offspring in [offspring1, offspring2]:
                        if hash(str(offspring)) not in unique_individuals:
                            next_generation.append(offspring)
                            unique_individuals.add(hash(str(offspring)))
                else:
                    for parent in [parent1, parent2]:
                        if hash(str(parent)) not in unique_individuals:
                            next_generation.append(parent)
                            unique_individuals.add(hash(str(parent)))

                # Step 6: Apply mutation to the next generation (except for best_individuals)
                if random.random() < self.mutation_rate:
                    for ind in next_generation:
                        if ind not in best_individuals:  # Avoid mutating the best individuals
                            self.mutate(ind)

            # Replace old population with the new generation
            self.population = next_generation[:self.population_size]  # Ensure population size remains constant

            for individual in self.population:
                print(individual.__str__(), individual.fitness_value)

            # Print the best individual of this generation
            best_individual = min(self.population, key=lambda x: x.fitness_value)
            print(f"Generation {current_generation + 1}: Best Fitness = {best_individual.fitness_value}")
            print(f"Generation {current_generation + 1}: Best MSE Percentage = {best_individual.mse_percentage}")

            # Adaptive mutation rate based on the mse percentage
            if best_individual.mse_percentage > 20:
                self.mutation_rate = 0.5
            else:
                self.mutation_rate = 0.2

            # Dynamically set tournament size based on MSE percentage
            if best_individual.mse_percentage > 20:  # High error, prioritize exploration
                tournament_size = 2
            elif 10 <= best_individual.mse_percentage <= 20:  # Moderate error, balance exploration and exploitation
                tournament_size = 3
            else:  # Low error, prioritize exploitation
                tournament_size = 5

            # Check if additional generations are needed
            if current_generation == total_generations-1 and best_individual.mse_percentage > 15:
                print("Extending the evolution process by 20 generations.")
                total_generations += 20

            # Stop evolution if the maximum allowed generations are reached
            if current_generation >= 90:
                print("Maximum generation limit (90) reached. Stopping evolution.")
                break

            # Stop if a nice solution is found
            if best_individual.mse_percentage < 10:  # Good: 5-10%, acceptable: 10-30%, poor: >30%
                print("Nice solution found")

            print(f"Best Individual: {best_individual.__str__()}")
            print("Nodes: ", best_individual.count_nodes(best_individual.root))

            current_generation += 1




# Usage example
file_path = 'problem_7.npz'  # Path to the problem_1.npz file

# Initialize the Symbolic Regression algorithm
sr = SymbolicRegression(
    population_size=200, 
    num_generations=50,
    mutation_rate=0.2,
    crossover_rate=0.7, 
    file_path=file_path
)

# Evolve the population
sr.evolve()


Starting Generation 1
Length of population of generation 1: 200


/var/folders/6l/pz7fgw6s2rqgl5c7dv0v9b780000gn/T/ipykernel_2607/3395420336.py:81: RuntimeWarning: overflow encountered in power
  result = np.power(base, exp)
/var/folders/6l/pz7fgw6s2rqgl5c7dv0v9b780000gn/T/ipykernel_2607/3395420336.py:81: RuntimeWarning: invalid value encountered in power
  result = np.power(base, exp)


(exp (x0 mul x1)) 625.9269517914341
(abs (abs (abs ((exp (x1 mul 0.8958333333333333)) add 6.833333333333333)))) 709.8544481940884
((cos (log2 (sinh (x0 sub 9.208333333333332)))) add 9.208333333333332) 711.3669852762166
((log (log2 (4.854166666666666 pow 2.4791666666666665))) mul 4.458333333333333) 717.454255420501
((sinh (8.020833333333332 pow (log2 (tanh 8.020833333333332)))) add 6.041666666666666) 720.2373629846523
(abs ((sin 4.854166666666666) add 6.041666666666666)) 737.9751730563786
(tan (log2 (cosh (tan (cosh (log10 0.8958333333333333)))))) 752.2634518806659
(x0 add (5.25 pow x0)) 761.3564891998872
((sqrt 9.604166666666666) add x1) 763.4946766836226
((tanh (0.8958333333333333 mul 2.4791666666666665)) mul (x0 mod 5.645833333333333)) 769.785852412437
((cosh (sinh x1)) mod (sqrt (x1 add x0))) 769.8185677781029
(exp (sin (log ((log 10.0) add (cos (cos 5.25)))))) 770.9720143941429
(2.4791666666666665 mul (log10 (sqrt x0))) 771.2304450769549
(exp (cos (sin (log10 x0)))) 774.16365312411